In [62]:
pip install pandas reverse_geocoder pycountry

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 4.1 MB/s eta 0:00:02
   ------------------------------- -------- 5.0/6.3 MB 15.6 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 15.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [49]:
# Import the necessary libraries
import pandas as pd
import numpy as np
# (https://github.com/thampiman/reverse-geocoder) to get the location names from latitude and longitude (faster then geopy, no hebrew names though...)
import reverse_geocoder as rg
# to get the full country name from country code
import pycountry


In [50]:
# lets take a look at the data
EQ_recent_history = pd.read_csv("D:\\Projects\\Tableau\\My Creations\\Israel's Earthquakes\\EQ_1981_2025-08-03.csv")
EQ_recent_history.head()

,epiid,DateTime,Mag,Lat,Long,Depth(Km),Region,Type
0,'202508010821',2025-08-01T08:21:53.912,2.9,27.8478,34.3057,67.903646,Arnona-Dakar-Deep,EQ
1,'202507280231',2025-07-28T02:31:46.262,4.0,34.7848,37.7575,30.761719,Palmira,EQ
2,'202507270723',2025-07-27T07:23:11.476,2.9,34.8710,33.6320,18.684896,Cyprus,EQ
3,'202507260912',2025-07-26T09:12:44.747,3.2,33.3385,33.0876,21.127563,NaN,EQ
4,'202507250331',2025-07-25T03:31:44.039,3.6,35.2748,32.6186,12.532552,NaN,EQ


In [56]:
def clean_recent_EQ(df):
    # Drop uneeded region column
    df = df.drop(columns=['Region'])
    
    # Rename columns for clarity
    df = df.rename(columns={
        'DateTime': 'date-time',
        'Mag': 'magnitude',
        'Lat': 'latitude',
        'Long': 'longitude',
        'Depth(Km)': 'depth',
        'Type': 'felt?'
    })

    # Strip leading and trailing whitespace in column: 'epiid'
    df['epiid'] = df['epiid'].str.strip("'")

    # Clean up the 'felt?' column
    df['felt?'] = df['felt?'].str.strip()
    df['felt?'] = df['felt?'].map({'EQ': False, 'F': True})

    # Format the 'date-time' field
    df['date-time'] = df['date-time'].str.replace('T', ' ')
    df['date-time'] = pd.to_datetime(df['date-time'])
    df['date-time'] = df['date-time'].dt.strftime('%d/%m/%Y %H:%M:%S')

    # Split 'date-time' into separate 'date' and 'time' columns
    df['date'] = df['date-time'].str.split(' ').str[0]

    # Reorder columns
    df = df[[
        "epiid", "latitude", "longitude", "date", "date-time", 
        "magnitude", "depth", "felt?"
    ]]
    
    return df

In [52]:
# Create a function to georeverse geocode the coordinates in the dataframe
def batch_reverse_geocode(df):
    # Create list of coordinate tuples
    coordinates = list(zip(df['latitude'], df['longitude']))
    
    # Pass the coordinate tuples to reverse geocoder to get results for all coordinates
    results = rg.search(coordinates)
    
    # Extract required fields into separate lists (results is a list of dictionaries)
    cities = [r['name'] for r in results]
    areas = [r['admin1'] for r in results]
    country_codes = [r['cc'] for r in results]

    # Convert country codes to full names
    def get_country_name(code):
        try:
            return pycountry.countries.get(alpha_2=code).name
        except AttributeError:
            return code  # Return original code if conversion fails
    
    countries = [get_country_name(code) for code in country_codes]
    
    # Add new columns to dataframe
    df['city'] = cities
    df['area'] = areas
    df['country'] = countries

    # Strip trailing backticks or single quotes from the city names 
    df['city'] = df['city'].str.lstrip("`'")
    # Remove ',' and text after it in the 'country' column
    df['country'] = df['country'].str.split(',').str[0]

    return df

In [57]:
# Clean & Apply the geocoding to our cleaned dataframe
EQ_recent_history_clean =  clean_recent_EQ(EQ_recent_history.copy())
EQ_recent_history_clean = batch_reverse_geocode(EQ_recent_history_clean)
EQ_recent_history_clean.head()

,epiid,latitude,longitude,date,date-time,magnitude,depth,felt?,city,area,country
0,202508010821,27.8478,34.3057,01/08/2025,01/08/2025 08:21:53,2.9,67.903646,False,Sharm el-Sheikh,South Sinai,Egypt
1,202507280231,34.7848,37.7575,28/07/2025,28/07/2025 02:31:46,4.0,30.761719,False,Uqayribat,Hama,Syrian Arab Republic
2,202507270723,34.8710,33.6320,27/07/2025,27/07/2025 07:23:11,2.9,18.684896,False,Meneou,Larnaka,Cyprus
3,202507260912,33.3385,33.0876,26/07/2025,26/07/2025 09:12:44,3.2,21.127563,False,Kolossi,Larnaka,Cyprus
4,202507250331,35.2748,32.6186,25/07/2025,25/07/2025 03:31:44,3.6,12.532552,False,Kato Pyrgos,Lefkosia,Cyprus


In [ ]:
#looks OK! let's save it to a CSV file for Tableau
EQ_recent_history_clean.to_csv("D:\\Projects\\Tableau\\My Creations\\Israel's Earthquakes\\all_EQ_cleaned.csv", index=False)